In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

Instructions for updating:
Use the retry module or similar alternatives.


In [2]:
mnist = input_data.read_data_sets("MNIST_data/",one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
# parameters

image_size = mnist.train.images.shape[1]  # 784
image_dim = int(image_size ** 0.5)  # 28
num_labels = 10
color_channels = 1

In [4]:
# placeholders

# Placeholder for the incoming 1D data 
x = tf.placeholder(dtype=tf.float32,shape=[None,image_size])

# Convolutional network accepts 2D data. So resize it from 1,784 to 28x28
x_image = tf.reshape(x,[-1,image_dim,image_dim,color_channels])

# Placeholder for the labels (one hot encoded)
y_true = tf.placeholder(dtype=tf.float32,shape=[None,num_labels])

## Graph creation

conv --> max pool --> conv --> max pool --> flatten --> fully connected --> drop out --> fully connected (output)

In [5]:
convo_layer_1 = tf.layers.conv2d(inputs=x_image,
                                 filters=32,
                                 kernel_size=[5,5],
                                 padding="same",
                                 activation=tf.nn.relu)

In [6]:
pool_layer_1 = tf.layers.max_pooling2d(inputs=convo_layer_1,
                                       pool_size=[2,2],
                                       strides=[2,2],
                                       padding="same")

In [7]:
convo_layer_2 = tf.layers.conv2d(inputs=pool_layer_1,
                                 filters=64,
                                 kernel_size=[5,5],
                                 padding="same",
                                 activation=tf.nn.relu)

In [8]:
pool_layer_2 = tf.layers.max_pooling2d(inputs=convo_layer_2,
                                       pool_size=[2,2],
                                       strides=[2,2],
                                       padding="same")

In [9]:
# Flattening the layer before sending it into the fully connected layer
flat_layer = tf.layers.flatten(pool_layer_2)

In [10]:
fully_connected_layer_1 = tf.contrib.layers.fully_connected(inputs=flat_layer,num_outputs=1024)

In [11]:
# creating a dropout layer which randomly drops neurons with probability = 0.5
# this is useful to avoid over fitting of data

drop_prob = tf.placeholder(dtype=tf.float32)
dropout_layer = tf.layers.dropout(inputs=fully_connected_layer_1,rate=drop_prob)

In [12]:
# this is the final output data 

output_layer = tf.contrib.layers.fully_connected(inputs=dropout_layer,num_outputs=num_labels,activation_fn=tf.nn.softmax)

#  Graph creation completed
### moving onto creating functions (cost) and the session

In [13]:
loss = tf.nn.softmax_cross_entropy_with_logits_v2(labels = y_true, logits = output_layer)
cost = tf.reduce_mean(loss)
optimizer = tf.train.AdamOptimizer()
train = optimizer.minimize(cost)

In [14]:
# initializing all the variables

init = tf.global_variables_initializer()

In [15]:
with tf.Session() as sess :
    sess.run(init)
    
    for i in range(5000):
        batch_x, batch_y = mnist.train.next_batch(batch_size=50)
        sess.run(train,feed_dict={x:batch_x,y_true:batch_y,drop_prob:0.5})
        if i%100 == 0 :
            matches = tf.equal(tf.argmax(output_layer,1),tf.argmax(y_true,1))
            accuracy = tf.reduce_mean(tf.cast(matches,dtype=tf.float32))
            res = sess.run(accuracy,feed_dict={x:mnist.test.images,y_true:mnist.test.labels,drop_prob:0})
            print("Step : %s" % i)
            print(res)
            print()

Step : 0
0.1315

Step : 100
0.8359

Step : 200
0.9455

Step : 300
0.9603

Step : 400
0.9689

Step : 500
0.9685

Step : 600
0.9639

Step : 700
0.9647

Step : 800
0.9753

Step : 900
0.9721

Step : 1000
0.9774

Step : 1100
0.9783

Step : 1200
0.9816

Step : 1300
0.98

Step : 1400
0.9774

Step : 1500
0.9747

Step : 1600
0.982

Step : 1700
0.9711

Step : 1800
0.9826

Step : 1900
0.9765

Step : 2000
0.9641

Step : 2100
0.9838

Step : 2200
0.9725

Step : 2300
0.9739

Step : 2400
0.9739

Step : 2500
0.9813

Step : 2600
0.9816

Step : 2700
0.9855

Step : 2800
0.9843

Step : 2900
0.9843

Step : 3000
0.9823

Step : 3100
0.9835

Step : 3200
0.9837

Step : 3300
0.9828

Step : 3400
0.9811

Step : 3500
0.9841

Step : 3600
0.9774

Step : 3700
0.9837

Step : 3800
0.9842

Step : 3900
0.9821

Step : 4000
0.9814

Step : 4100
0.9818

Step : 4200
0.9823

Step : 4300
0.9786

Step : 4400
0.985

Step : 4500
0.9693

Step : 4600
0.9831

Step : 4700
0.9856

Step : 4800
0.9831

Step : 4900
0.9813



In [ ]:
# the final accuracy on the test set is around 98 %